# System Architecture and Communication Topology
---

In this notebook, we demonstrated how to explore system topology and intra-node/inter-node communication concepts. We further delve into NVIDIA Collective Communication Library (NCCL), detailing five collective communication primitives that include `AllReduce,` `Broadcast,` `Reduce,` `AllGather,` and `ReduceScatter.`  

## Important Terminology

- **Host**: The CPU and its memory (host memory)
- **Device**: The GPU and its memory (device memory)
- **Kernel**:  A GPU function launched by the host and executed on the device.
- **Latency:** The amount of time it takes to take a unit of data from point A to point B. For example, if 4B of data can be transferred from point A to B in 4 $\mu$s, that is the transfer latency.
- **Bandwidth:** The amount of data that can be transferred from point A to point B in a unit of time. For example, if the width of the bus is 64KB and the latency of transfer between points A and B is 4 $\mu$s, the bandwidth is 64KB * (1/4$\mu$s) = 1.6 GB/s.


## Communication Approaches   

There are several GPU communication approaches that involve data copying/movement operations. Let’s look at two common basic concepts: Host Staging of Copy Operations and Peer-to-Peer Memory Access. Understanding these concepts will lay a foundation for more complex approaches to be demonstrated in the rest of this content.

### I. Host Staging of Copy Operations

The diagram below illustrates data movement (following the red arrow path) from GPU 0 to GPU 1 in a GPU-to-GPU memory copy operation. The data moves from GPU 0 and passes through the PICe bus to the CPU, where it is staged in a buffer before being copied to GPU 1. This process is called host stagging, which thus reduces bandwidth and increases latency. Process performance can be improved by eliminating host staging.

<img src="images/DL_host_staging.png" width="380px" height="380px" alt-text="Host"/>


### II. Peer-to-Peer Memory Access

The P2P approach allows GPUs to address each other's memory from within device kernels and eliminates host staging by transferring data either through the PCIe switch or through NVLink, as denoted by the red arrows in the diagram below. For example,  data can be transversed directly from GPU 0 to GPU 1 via NVLink or from GPU 2 to GPU 3 via the PCIe. The P2P approach allows GPUs to address each other's memory from within device kernels and eliminates host staging by transferring data either through the PCIe switch or through NVLink, as denoted by the red arrows in the diagram below. For example,  data can be transversed directly from GPU 0 to GPU 1 via NVLink or from GPU 2 to GPU 3 via the PCIe. This approach requires GPUs to share a Unified Virtual Address Space (UVA). UVA means that single address space is used for the host and all modern NVIDIA GPU devices (specifically, those with a compute capability of 2.0 or higher).

<img src="images/DL_p2p.png" width="380px" height="380px" alt-text="p2p"/>


Let's check if P2P is supported between the GPUs by running the command in the cell below:

In [ ]:
!srun --partition=gpu -n1 --gres=gpu:4 nvidia-smi topo -p2p r

**Likely output on DGX H100**:

<img src="images/H100-p2p-r.png" width="380px" height="380px" alt-text="p2pr"/>

This means all the GPUs can communicate via P2P with each other through PCIe (You will also find similar results in DGX A100). To check whether P2P via NVLink is supported, please run the command below:

In [ ]:
!srun --partition=gpu -n1 --gres=gpu:4 nvidia-smi topo -p2p n

**Likely output on DGX H100**:

<img src="images/H100-p2p-n.png" width="380px" height="380px" alt-text="p2pn"/>

This implies that all the GPUs can communicate with each other in P2P via NVLink. Therefore, host staging is eliminated, and performance is improved. 
The `topo` sub-command requests information on the GPU communication topology. The -p2p or –p2p2status flag displays the p2p status between the GPUs of a given p2p capability as listed below:

- **r**: p2p read capability
- **w**: p2p write capability
- **n**: p2p nvlink capability
- **a**: p2p atomic capability
- **p**: p2p prop capability 


## Intra-Node Commumications

This section will consider DGX A100 and H100 communication architecture and the Network Interface Card (NIC) topology.

### DGX A100 Topology

Run the command `!nvidia-smi topo -m` in the cell below to display your node’s GPU and  NIC communication topology.


In [ ]:
!srun --partition=gpu -n1 --gres=gpu:4 nvidia-smi topo -m

**Expected Output**
<center><img src="images/A100-terminal-topo.png" width="850px" height="750px" alt-text="Arc"/></center>


GPU0 to GPU7 are connected via NVLink(NV12), and in summary, each GPU in A100 is connected to all other GPUs via the third-generation NVLink. As a result, this doubles the GPU-to-GPU direct bandwidth to 600 gigabytes per second (GB/s) compared to V100. You can also test the command `nvidia-smi topo -mp` to display the GPUDirect communication matrix via only PCIe for the system. 

The diagram below shows the high-level topology overview of DGX A100. Within a node, each of the 8 GPUs can efficiently communicate with each other via NVLink Switch (NVSwitch).

<center><img src="images/dgx_a100_arch_topo.png" width="700px" height="700px" alt-text="Arc"/></center>



#### Streaming Multiprocessors (SMs)

The A100 SM includes new third-generation Tensor Cores that each perform 256 FP16/FP32 mixed-precision fused multiply-add (FMA) operations per clock. A100 has four Tensor Cores per SM, which together deliver 1024 dense FP16/FP32 FMA operations per clock. Key SM features on Third-generation Tensor Cores include:

- Acceleration for all data types including FP16, BF16, TF32, FP64, INT8, INT4, and Binary.
- New Tensor Core sparsity feature exploits fine-grained structured sparsity in deep learning networks, doubling the performance of standard Tensor Core operations. The NVIDIA A100 GPU supports fine-grained structured sparsity to accelerate simplified neural networks without harming accuracy. Sparsity often comes from pruning - the technique of removing weights that contribute little to the accuracy of the network. Typically, this involves "zeroing out" and removing weights that have zero or near-zero values. 

 <center><img src="images/dgx-A100-sparsity-grained.png" width="700px" height="700px" alt-text="Arc"/></center>

- TF32 Tensor Core operations in A100 provide an easy path to accelerate FP32 input/output data in DL frameworks. 
- FP16/FP32 mixed-precision Tensor Core operations deliver unprecedented processing power for DL. 
- BF16/FP32 mixed-precision Tensor Core operations run at the same rate as FP16/FP32 mixed-precision.
- FP64 Tensor Core operations deliver unprecedented double precision processing power for HPC. 
- INT8 Tensor Core operations with sparsity deliver unprecedented processing power for DL Inference, running up to 20x faster than V100 INT8 operations.


### DGXC H100 Topology

The NVIDIA H100 GPU based on the new Hopper GPU architecture features multiple innovations:
- New fourth-generation Tensor Cores perform faster matrix computations than ever before on an even broader array of AI and HPC tasks.
- A new Transformer Engine enables H100 to deliver up to 9x faster AI training and up to 30x faster AI inference speedups on large language models compared to the prior generation A100.
- The new NVLink Network interconnect enables GPU-to-GPU communication among up to 256 GPUs across multiple compute nodes
- Secure MIG partitions the GPU into isolated, right-size instances to maximize QoS (quality of service) for smaller workloads. 

Run the command `!nvidia-smi topo -m` in the cell below to display your node’s GPU and  NIC communication topology.

In [ ]:
!srun --partition=gpu -n1 --gres=gpu:4 nvidia-smi topo -m

**Expected Output**:

<center><img src="images/dgx-h100-terminal-topo.png" width="850px" height="750px" alt-text="h100-topo-terminal"/></center>


GPU0 to GPU7 are connected via NVLink(NV18), and in summary, each GPU in H100 is connected to all other GPUs via the fourth-generation NVLink. As a result, this increases the GPU-to-GPU direct bandwidth to 900 gigabytes per second (GB/s) compared to A100. You can also test the command `nvidia-smi topo -mp` to display the GPUDirect communication matrix via only PCI for the system.
The diagram below shows the high-level topology overview of DGX H100. Within a node, each of the 8 GPUs can efficiently communicate with each other via NVLink Switch (NVSwitch).

<center><img src="images/dgxh100-topo.png" width="700px" height="700px" alt-text="Arc"/></center>

 


 #### H100 SM Architecture 

Building upon the NVIDIA A100 Tensor Core GPU SM architecture, the H100 SM quadruples A100’s peak per-SM floating point computational power due to the introduction of FP8 and doubles A100’s raw SM computational power on all previous Tensor Core and FP32 / FP64 data types, clock-for-clock. Hopper’s new fourth-generation Tensor Core, Tensor Memory Accelerator, and many other new SM and general H100 architecture improvements together deliver up to 3x faster HPC and AI performance in many other cases.   

<center>NVIDIA H100 Tensor Core GPU Performance Specs</center>
<center><img src="images/H100-tensor-core-gpu-performance-spec.png" width="500px" height="500px" alt-text="Arc"/></center>

Key SM features on Fourth-generation Tensor Cores include:
- Up to 6x faster chip-to-chip compared to A100, including per-SM speedup, additional SM count, and higher clocks of H100.
- On a per SM basis, the Tensor Cores deliver 2x the MMA (Matrix Multiply Accumulate) computational rates of the A100 SM on equivalent data types and 4x the rate of A100 using the new FP8 data type, compared to previous generation 16-bit floating-point options.
- Sparsity feature exploits fine-grained structured sparsity in deep learning networks, doubling the performance of standard Tensor Core operations 

#### Hopper FP8 Data Format
The H100 GPU adds FP8 Tensor Cores to accelerate both AI training and inference. As shown in the diagram below, FP8 Tensor Cores support FP32 and FP16 accumulators, and two new FP8 input types:
- E4M3 with 4 exponent bits, 3 mantissa bits, and 1 sign bit
- E5M2, with 5 exponent bits, 2 mantissa bits, and 1 sign bit.

E4M3 supports computations requiring less dynamic range with more precision, while E5M2 provides a wider dynamic range and less precision. FP8 halves data storage requirements and doubles throughput compared to FP16 or BF16. 
New Hopper FP8 Precisions - 2x throughput and half the footprint of H100 FP16 / BF16. H100 FP8 Tensor Core 6x throughput compared to A100 FP16 Tensor Core. 

<center><img src="images/H100-FP8-precision-1.png" width="500px" height="500px" alt-text="Arc"/></center>


## NVIDIA Collective Communication Library (NCCL)

The [NVIDIA Collective Communications Library](https://developer.nvidia.com/nccl) is a library providing inter-GPU communication primitives that are topology-aware and can be easily integrated into applications. NCCL implements both collective communication and point-to-point send/receive primitives. It is not a full-blown parallel programming framework but a library focused on accelerating inter-GPU communication. NCCL provides the following collective communication primitives:

- **AllReduce**: The AllReduce operation performs reductions on data (for example, sum, min, max) across devices and stores the result in the receive buffer of every rank. In a sum `allreduce` operation between k ranks, each rank will provide an array in of N values and receive identical results in an array out of N values, where `out[i] = in0[i]+in1[i]+…+in(k-1)[i]`.

<center><img src="images/allreduce.png" width="380px" height="380px" alt-text="allreduce"/> </center>


- **Broadcast**: The Broadcast operation copies an N-element buffer from the root rank to all the ranks.

<center><img src="images/broadcast.png" width="380px" height="380px" alt-text="broadcast"/> </center>

- **Reduce**: The Reduce operation performs the same operation as `AllReduce`, but stores the result only in the receive buffer of a specified root rank. A Reduce, followed by a Broadcast, is equivalent to the AllReduce operation.

  <center><img src="images/reduce.png" width="380px" height="380px" alt-text="reduce"/> </center>

- **AllGather**: The AllGather operation gathers N values from k ranks into an output buffer of size `k*N` and distributes that result to all ranks. The output is ordered by the rank index. The AllGather operation is, therefore, impacted by a different rank to device mapping. Executing ReduceScatter, followed by AllGather, is equivalent to the AllReduce operation.

<center><img src="images/allgather.png" width="380px" height="380px" alt-text="allgather"/> </center>

- **ReduceScatter**: The ReduceScatter operation performs the same operation as Reduce, except that the result is scattered in equal-sized blocks between ranks, each rank getting a chunk of data based on its rank index. The ReduceScatter operation is impacted by a different rank-to-device mapping since the ranks determine the data layout.

  <center><img src="images/reducescatter.png" width="380px" height="380px" alt-text="reducescatter"/> </center>


Additionally, it allows for point-to-point send/receive communication which allows for scatter, gather, or all-to-all operations.
NCCL conveniently removes the need for developers to optimize their applications for specific machines. It provides fast collectives over multiple GPUs both within and across nodes. It supports a variety of interconnect technologies, including PCIe, NVLINK, InfiniBand Verbs, and IP sockets. NCCL has found great application in Deep Learning Frameworks, where the AllReduce collective is heavily used for neural network training. Efficient scaling of neural network training is possible with the multi-GPU and multi-node communication provided by NCCL.



#### Environment Variables

NCCL has an extensive set of [environment variables](https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/env.html) to tune for specific usage.  There are two categories of environment variables: system configuration and Debugging. From the debugging category, let’s look at the environment variable for P2P:

- **NCCL_P2P_DISABLE**: The NCCL_P2P_DISABLE variable disables the peer-to-peer (P2P) transport, which uses CUDA direct access between GPUs, using NVLink or PCI. The value should be set to 1 to disable direct GPU-to-GPU (P2P) communication.
- **NCCL_P2P_LEVEL**: The NCCL_P2P_LEVEL variable allows the user to finely control when to use the peer-to-peer (P2P) transport between GPUs. The level defines the maximum distance between GPUs where NCCL will use the P2P transport. A short string representing the path type should be used to specify the topographical cutoff for using the P2P transport. If this isn’t specified, NCCL will attempt to optimally select a value based on the architecture and environment in which it’s run.

**Acceptable values and legacy integers**
```text
LOC or 0: Never use P2P (always disabled)
NVL : Use P2P when GPUs are connected through NVLink
PIX or 1: Use P2P when GPUs are on the same PCI switch.
PXB or 2: Use P2P when GPUs are connected through PCI switches (potentially multiple hops).
PHB or 3: Use P2P when GPUs are on the same NUMA node. Traffic will go through the CPU.
SYS or 4: Use P2P between NUMA nodes, potentially crossing the SMP interconnect (e.g. QPI/UPI).
```


#### NCCL_P2P_LEVEL=0 or P2P Disabled

To disable NCCL P2P, please set these variables:`NCCL_P2P_LEVEL=0 NCCL_DEBUG=INFO CUDA_VISIBLE_DEVICES="0,1"` before the `torchrun` command as follows: `NCCL_P2P_LEVEL=0 NCCL_DEBUG=INFO CUDA_VISIBLE_DEVICES="0,1" torchrun --nnodes=1 --nproc-per-node=2 sample_code.py.` Because of the way the commands in this content were designed, we have initialized the variable within our code. 

```python
# Setting environment variables for NCCL
os.environ["NCCL_P2P_LEVEL"]="0"
os.environ["NCCL_DEBUG"]="INFO"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
```
Please run the cell below

In [ ]:
!cd ../source_code && srun --partition=gpu -n1 --gres=gpu:2 torchrun --nnodes=1 --nproc-per-node=2  train_ddp.py

**Likely Output on DGX A100:**

```python
...
dgx01:2366722:2366722 [1] NCCL INFO NET/Plugin: Using internal network plugin.
dgx01:2366722:2366722 [1] NCCL INFO Comm config Blocking set to 1
dgx01:2366721:2366799 [0] NCCL INFO NET/IB : Using [0]mlx5_0:1/IB [1]mlx5_1:1/IB [2]mlx5_2:1/IB [3]mlx5_3:1/IB [4]mlx5_4:1/IB [5]mlx5_5:1/RoCE [6]mlx5_6:1/IB [7]mlx5_7:1/IB [8]mlx5_8:1/IB [9]mlx5_9:1/IB [10]mlx5_10:1/IB [RO]; OOB ibp12s0:100.126.5.1<0>
dgx01:2366721:2366799 [0] NCCL INFO Using non-device net plugin version 0
dgx01:2366721:2366799 [0] NCCL INFO Using network IB
dgx01:2366722:2366801 [1] NCCL INFO NET/IB : Using [0]mlx5_0:1/IB [1]mlx5_1:1/IB [2]mlx5_2:1/IB [3]mlx5_3:1/IB [4]mlx5_4:1/IB [5]mlx5_5:1/RoCE [6]mlx5_6:1/IB [7]mlx5_7:1/IB [8]mlx5_8:1/IB [9]mlx5_9:1/IB [10]mlx5_10:1/IB [RO]; OOB ibp12s0:100.126.5.1<0>
dgx01:2366722:2366801 [1] NCCL INFO Using non-device net plugin version 0
dgx01:2366722:2366801 [1] NCCL INFO Using network IB
...
dgx01:2366721:2366799 [0] NCCL INFO ncclCommInitRank comm 0x750be00 rank 0 nranks 2 cudaDev 0 nvmlDev 0 busId 7000 commId 0xfce657e6872b563c - Init COMPLETE
dgx01:2366722:2366801 [1] NCCL INFO ncclCommInitRank comm 0xdfb4c50 rank 1 nranks 2 cudaDev 1 nvmlDev 1 busId f000 commId 0xfce657e6872b563c - Init COMPLETE
100%|██████████| 196/196 [01:15<00:00,  2.58it/s]
Epoch 1 Time 75.911103900522
100%|██████████| 196/196 [01:15<00:00,  2.58it/s]
100%|██████████| 196/196 [01:15<00:00,  2.59it/s]
Epoch 2 Time 75.67138409987092

```

#### NCCL_P2P_LEVEL=1 or P2P via PCIe
To enable NCCL P2P via the PCIe, set these variables:`NCCL_P2P_LEVEL=1 NCCL_DEBUG=INFO CUDA_VISIBLE_DEVICES="0,1".` To effect this, please modify line #18 within our [train_ddp.py](../source_code/train_ddp.py) as shown below.

```python
# Setting environment variables for NCCL
os.environ["NCCL_P2P_LEVEL"]="1"
```
Please run the cell below to see NCCL P2P via PCIe traces. 

In [ ]:
!cd ../source_code && srun --partition=gpu -n1 --gres=gpu:2 torchrun --nnodes=1 --nproc-per-node=2  train_ddp.py

**Likely Output on A100:**

```python
...
Init START
dgx01:2311707:2311808 [0] NCCL INFO NCCL_P2P_LEVEL set by environment to PIX
dgx01:2311707:2311808 [0] NCCL INFO Setting affinity for GPU 0 to ffff0000,00000000,00000000,00000000,ffff0000,00000000
dgx01:2311708:2311810 [1] NCCL INFO NCCL_P2P_LEVEL set by environment to PIX
dgx01:2311708:2311810 [1] NCCL INFO Setting affinity for GPU 1 to ffff0000,00000000,00000000,00000000,ffff0000,00000000
dgx01:2311708:2311810 [1] NCCL INFO comm 0xdfbcf30 rank 1 nRanks 2 nNodes 1 localRanks 2 localRank 1 MNNVL 0
...
dgx01:2311707:2311808 [0] NCCL INFO Channel 22/24 :    0   1
dgx01:2311707:2311808 [0] NCCL INFO Channel 23/24 :    0   1
dgx01:2311707:2311808 [0] NCCL INFO Trees [0] 1/-1/-1->0->-1 [1] 1/-1/-1->0->-1 [2] 1/-1/-1->0->-1 [3] 1/-1/-1->0->-1 [4] 1/-1/-1->0->-1 [5] 1/-1/-1->0->-1 [6] -1/-1/-1->0->1 [7] -1/-1/-1->0->1 [8] -1/-1/-1->0->1 [9] -1/-1/-1->0->1 [10] -1/-1/-1->0->1 [11] -1/-1/-1->0->1 [12] 1/-1/-1->0->-1 [13] 1/-1/-1->0->-1 [14] 1/-1/-1->0->-1 [15] 1/-1/-1->0->-1 [16] 1/-1/-1->0->-1 [17] 1/-1/-1->0->-1 [18] -1/-1/-1->0->1 [19] -1/-1/-1->0->1 [20] -1/-1/-1->0->1 [21] -1/-1/-1->0->1 [22] -1/-1/-1->0->1 [23] -1/-1/-1->0->1
dgx01:2311707:2311808 [0] NCCL INFO P2P Chunksize set to 524288
dgx01:2311708:2311810 [1] NCCL INFO Channel 00/0 : 1[1] -> 0[0] via P2P/CUMEM/read
dgx01:2311708:2311810 [1] NCCL INFO Channel 01/0 : 1[1] -> 0[0] via P2P/CUMEM/read
dgx01:2311708:2311810 [1] NCCL INFO Channel 02/0 : 1[1] -> 0[0] via P2P/CUMEM/read
dgx01:2311707:2311808 [0] NCCL INFO Channel 00/0 : 0[0] -> 1[1] via P2P/CUMEM/read
dgx01:2311708:2311810 [1] NCCL INFO Channel 03/0 : 1[1] -> 0[0] via P2P/CUMEM/read
...
gx01:2311707:2311808 [0] NCCL INFO TUNER/Plugin: Using internal tuner plugin.
dgx01:2311707:2311808 [0] NCCL INFO ncclCommInitRank comm 0x750be80 rank 0 nranks 2 cudaDev 0 nvmlDev 0 busId 7000 commId 0x20bbb052df5c1aea - Init COMPLETE
100%|██████████| 196/196 [01:14<00:00,  2.62it/s]
100%|██████████| 196/196 [01:14<00:00,  2.62it/s]
Epoch 1 Time 74.94601937010884
100%|██████████| 196/196 [01:15<00:00,  2.60it/s]
100%|██████████| 196/196 [01:15<00:00,  2.60it/s]
Epoch 2 Time 75.33938680775464
...
```

*Note: NCCL P2P via NVLink is by default in Ampere and Hopper DGX Architectures*

Now that we have a fundamental understanding of system topology, we can delve into distributed training strategy. Please proceed to the `Distributed Data Parallelism` notebook using the [Next Link](data-parallelism.ipynb). 

---
## References

- https://www.nvidia.com/en-us/data-center/nvlink/
- https://developer.nvidia.com/blog/nvidia-nvlink-and-nvidia-nvswitch-supercharge-large-language-model-inference
- https://images.nvidia.com/aem-dam/en-zz/Solutions/data-center/nvidia-ampere-architecture-whitepaper.pdf
- https://medium.com/red-buffer/getting-started-with-pytorch-distributed-54ae933bb9f0
- https://developer.nvidia.com/nsight-systems/get-started

## Licensing

Copyright © 2025 OpenACC-Standard.org. This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.